## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,30.20,86,40,4.61,scattered clouds
1,1,Anadyr,RU,64.7500,177.4833,59.50,38,0,7.29,clear sky
2,2,Buin,CL,-33.7333,-70.7500,64.06,41,100,2.64,overcast clouds
3,3,Upernavik,GL,72.7868,-56.1549,31.33,94,100,4.32,overcast clouds
4,4,Hobart,AU,-42.8794,147.3294,46.99,76,0,9.22,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Anadyr,RU,64.7500,177.4833,59.50,38,0,7.29,clear sky
7,7,Raesfeld,DE,51.7667,6.8500,57.79,95,0,6.42,clear sky
14,14,Port Alfred,ZA,-33.5906,26.8910,56.73,67,100,13.73,overcast clouds
22,22,Busselton,AU,-33.6500,115.3333,54.09,65,100,14.27,overcast clouds
25,25,Port Lincoln,AU,-34.7333,135.8667,58.55,37,100,16.46,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Anadyr,RU,64.7500,177.4833,59.50,38,0,7.29,clear sky
7,7,Raesfeld,DE,51.7667,6.8500,57.79,95,0,6.42,clear sky
14,14,Port Alfred,ZA,-33.5906,26.8910,56.73,67,100,13.73,overcast clouds
22,22,Busselton,AU,-33.6500,115.3333,54.09,65,100,14.27,overcast clouds
25,25,Port Lincoln,AU,-34.7333,135.8667,58.55,37,100,16.46,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Anadyr,RU,59.50,clear sky,64.7500,177.4833,
7,Raesfeld,DE,57.79,clear sky,51.7667,6.8500,
14,Port Alfred,ZA,56.73,overcast clouds,-33.5906,26.8910,
22,Busselton,AU,54.09,overcast clouds,-33.6500,115.3333,
25,Port Lincoln,AU,58.55,overcast clouds,-34.7333,135.8667,
26,Burnie,AU,51.93,scattered clouds,-41.0667,145.9167,
27,Onguday,RU,53.13,scattered clouds,50.7500,86.1500,
31,Kaitangata,NZ,50.97,overcast clouds,-46.2817,169.8464,
32,Mataura,NZ,50.00,overcast clouds,-46.1927,168.8643,
35,Kincardine,GB,55.81,clear sky,56.0667,-3.7167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {"radius": 5000,"type": "lodging","key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"] 
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")
hotel_df.head(10) 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Anadyr,RU,59.50,clear sky,64.7500,177.4833,"Гостиница ""Анадырь"""
7,Raesfeld,DE,57.79,clear sky,51.7667,6.8500,Hotel am Tiergarten
14,Port Alfred,ZA,56.73,overcast clouds,-33.5906,26.8910,The Halyards Hotel
22,Busselton,AU,54.09,overcast clouds,-33.6500,115.3333,Observatory Guest House
25,Port Lincoln,AU,58.55,overcast clouds,-34.7333,135.8667,Port Lincoln Hotel
26,Burnie,AU,51.93,scattered clouds,-41.0667,145.9167,Beachfront Voyager Motor Inn
27,Onguday,RU,53.13,scattered clouds,50.7500,86.1500,"Gostinitsa ""Motel'"""
31,Kaitangata,NZ,50.97,overcast clouds,-46.2817,169.8464,Kaitangata Motor Camp
32,Mataura,NZ,50.00,overcast clouds,-46.1927,168.8643,Ellie's Villa
35,Kincardine,GB,55.81,clear sky,56.0667,-3.7167,Premier Inn Falkirk North hotel


In [9]:
# 7. Drop the rows where there is no Hotel Name.

import numpy as np

clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] !="")]
clean_hotel_df.head()

#hotel_df.loc[(hotel_df["Hotel Name"] == "")]=np.delete()
#test=clean_hotel_df.loc[9, 'Hotel Name']
#test
#print(df.replace(r'^\s*$', np.nan, regex=True))




,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Anadyr,RU,59.50,clear sky,64.7500,177.4833,"Гостиница ""Анадырь"""
7,Raesfeld,DE,57.79,clear sky,51.7667,6.8500,Hotel am Tiergarten
14,Port Alfred,ZA,56.73,overcast clouds,-33.5906,26.8910,The Halyards Hotel
22,Busselton,AU,54.09,overcast clouds,-33.6500,115.3333,Observatory Guest House
25,Port Lincoln,AU,58.55,overcast clouds,-34.7333,135.8667,Port Lincoln Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.????????why do we do this step
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
#fig.add_layer(heat_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))